In [1]:
#Import Library
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import accuracy_score


In [3]:
# Load and prepare data
loan_dataset = pd.read_csv('/home/parth/Downloads/FML Lab/Dataset/loanData.csv')
loan_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299 entries, 0 to 298
Data columns (total 30 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    299 non-null    int64  
 1   year                  299 non-null    int64  
 2   issue_d               299 non-null    object 
 3   final_d               299 non-null    int64  
 4   emp_length_int        299 non-null    float64
 5   home_ownership        299 non-null    object 
 6   home_ownership_cat    299 non-null    int64  
 7   income_category       299 non-null    object 
 8   annual_inc            299 non-null    int64  
 9   income_cat            299 non-null    int64  
 10  loan_amount           299 non-null    int64  
 11  term                  299 non-null    object 
 12  term_cat              299 non-null    int64  
 13  application_type      299 non-null    object 
 14  application_type_cat  299 non-null    int64  
 15  purpose               2

In [13]:
loan_dataset.head()
loan_dataset.shape

(299, 30)

In [5]:

loan_dataset.isnull().sum()
loan_dataset = loan_dataset.dropna()
loan_dataset = loan_dataset.replace(to_replace='3+', value=4)
loan_dataset.replace({'income_category':{'Low':0,'High':1},'home_ownership':{'RENT':1,'MORTGAGE':0},'grade':{'A':5,'B':4,'C':3,'D':2,'E':1,'F':0},'Property_Area':{'Rural':0,'Semiurban':1,'Urban':2},'Education':{'Graduate':1,'Not Graduate':0}},inplace=True)


In [6]:
loan_dataset.head(100)

,id,year,issue_d,final_d,emp_length_int,home_ownership,home_ownership_cat,income_category,annual_inc,income_cat,...,loan_condition_cat,interest_rate,grade,grade_cat,dti,total_pymnt,total_rec_prncp,recoveries,installment,region
0,1077501,2011,01/12/2011,1012015,10.0,1,1,0,24000,1,...,0,10.65,4,2,27.65,5861.071414,5000.00,0.00,162.87,munster
1,1077430,2011,01/12/2011,1042013,0.5,1,1,0,30000,1,...,1,15.27,3,3,1.00,1008.710000,456.46,117.08,59.83,leinster
2,1077175,2011,01/12/2011,1062014,10.0,1,1,0,12252,1,...,0,15.96,3,3,8.72,3003.653644,2400.00,0.00,84.33,cannught
3,1076863,2011,01/12/2011,1012015,10.0,1,1,0,49200,1,...,0,13.49,3,3,20.00,12226.302210,10000.00,0.00,339.31,ulster
4,1075358,2011,01/12/2011,1012016,1.0,1,1,0,80000,1,...,0,12.69,4,2,17.94,3242.170000,2233.10,0.00,67.79,ulster
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1068575,2011,01/12/2011,1012016,6.0,1,1,0,85000,1,...,0,22.06,0,6,20.58,20295.800000,10776.09,0.00,423.10,leinster
96,1049528,2011,01/12/2011,1112013,4.0,0,3,0,47800,1,...,0,11.71,4,2,10.04,15161.270000,12800.00,0.00,282.86,leinster
97,1068542,2011,01/12/2011,1042013,7.0,0,3,0,45000,1,...,1,17.27,2,4,21.31,10037.100000,3077.85,3486.00,437.47,ulster
98,1068350,2011,01/12/2011,1012015,10.0,0,3,0,83000,1,...,0,6.03,5,1,2.31,3834.660850,3500.00,0.00,106.53,cannught


In [7]:
# Identify predictor and target
X = loan_dataset.drop(columns=['loan_condition','interest_payments','term','purpose','application_type','region','grade','income_category','home_ownership','issue_d','id','loan_condition_cat'],axis=1)
Y = loan_dataset['loan_condition_cat']


In [8]:
X.head(100)

,year,final_d,emp_length_int,home_ownership_cat,annual_inc,income_cat,loan_amount,term_cat,application_type_cat,purpose_cat,interest_payment_cat,interest_rate,grade_cat,dti,total_pymnt,total_rec_prncp,recoveries,installment
0,2011,1012015,10.0,1,24000,1,5000,1,1,1,1,10.65,2,27.65,5861.071414,5000.00,0.00,162.87
1,2011,1042013,0.5,1,30000,1,2500,2,1,2,2,15.27,3,1.00,1008.710000,456.46,117.08,59.83
2,2011,1062014,10.0,1,12252,1,2400,1,1,3,2,15.96,3,8.72,3003.653644,2400.00,0.00,84.33
3,2011,1012015,10.0,1,49200,1,10000,1,1,4,2,13.49,3,20.00,12226.302210,10000.00,0.00,339.31
4,2011,1012016,1.0,1,80000,1,3000,2,1,4,1,12.69,2,17.94,3242.170000,2233.10,0.00,67.79
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2011,1012016,6.0,1,85000,1,15300,2,1,1,2,22.06,6,20.58,20295.800000,10776.09,0.00,423.10
96,2011,1112013,4.0,3,47800,1,20000,2,1,6,1,11.71,2,10.04,15161.270000,12800.00,0.00,282.86
97,2011,1042013,7.0,3,45000,1,17500,2,1,1,2,17.27,4,21.31,10037.100000,3077.85,3486.00,437.47
98,2011,1012015,10.0,3,83000,1,3500,1,1,2,1,6.03,1,2.31,3834.660850,3500.00,0.00,106.53


In [9]:
# Split train data and test data
X_train, X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.1,stratify=Y,random_state=2)


In [10]:
# Model classifier using linear svm
classifier = svm.SVC(kernel='linear')

In [11]:
classifier.fit(X_train,Y_train)

SVC(kernel='linear')

In [12]:
X_train_prediction = classifier.predict(X_train)
training_data_accuray = accuracy_score(X_train_prediction,Y_train)
print('Accuracy on training data : ', training_data_accuray)
X_test_prediction = classifier.predict(X_test)
test_data_accuray = accuracy_score(X_test_prediction,Y_test)
print('Accuracy on test data : ', test_data_accuray)

Accuracy on training data :  0.966542750929368
Accuracy on test data :  1.0
